In [2]:
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler


In [3]:
#Cargo los sets
X_train = pd.read_csv('data/X_train_ohc.csv', dtype={'id':'int',\
                                           'tipodepropiedad':'category',\
                                           'ciudad':'category', 'provincia':'category',\
                                           'antiguedad':'float', 'habitaciones':'float',\
                                           'garages':'float','banos':'float',\
                                           'metroscubiertos':'float', 'metrostotales':'float',\
                                           'idzona':'float',\
                                           'gimnasio':'bool', 'usosmultiples':'bool',\
                                           'piscina':'bool', 'escuelascercanas':'bool',\
                                           'centroscomercialescercanos':'bool'\
                                           }, index_col='id')
y_train = pd.read_csv('data/y_train.csv',header=None,squeeze=True,index_col=0)
X_test = pd.read_csv('data/X_test_ohc.csv', dtype={'id':'int',\
                                           'tipodepropiedad':'category',\
                                           'ciudad':'category', 'provincia':'category',\
                                           'antiguedad':'float', 'habitaciones':'float',\
                                           'garages':'float','banos':'float',\
                                           'metroscubiertos':'float', 'metrostotales':'float',\
                                           'idzona':'float',\
                                           'gimnasio':'bool', 'usosmultiples':'bool',\
                                           'piscina':'bool', 'escuelascercanas':'bool',\
                                           'centroscomercialescercanos':'bool'\
                                           }, index_col='id')
y_test = pd.read_csv('data/y_test.csv',header=None,squeeze=True,index_col=0)

In [4]:
features_n = pd.read_csv('data/features_importancia_primera_etapa.csv',index_col=0)\
             .head(70).index.tolist()
features_n[0:5]

['metroscubiertos',
 'metrostotales',
 'banos',
 'provincia__Distrito Federal',
 'idzona']

In [5]:
X_train_n = X_train[features_n]
X_test_n = X_test[features_n]

In [6]:
features_numericos = X_train_n.dtypes[X_train.dtypes != "object"].index
len(features_numericos)
#son todas numericas obviamente

70

In [7]:
scaler = StandardScaler()

scaled = scaler.fit_transform(X_train_n[features_numericos])
for i, col in enumerate(features_numericos):
    X_train_n[col] = scaled[:, i]

scaled = scaler.fit_transform(X_test_n[features_numericos])
for i, col in enumerate(features_numericos):
    X_test_n[col] = scaled[:, i]

/home/franco/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype bool, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/franco/anaconda3/lib/python3.7/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype bool, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/franco/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/home/franco/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype bool, int64, float64 were all converted to float

In [35]:
def RMSE(actual, pred):
    return np.sqrt(mean_squared_error(actual, pred))

In [36]:
y_train_log = np.log1p(y_train)
y_test_log = np.log1p(y_test)

In [37]:
best_alpha = 0.00099

regr = Lasso(alpha=best_alpha, max_iter=50000)
%time regr.fit(X_train_n, y_train_log)

CPU times: user 3.84 s, sys: 123 ms, total: 3.97 s
Wall time: 1 s


Lasso(alpha=0.00099, copy_X=True, fit_intercept=True, max_iter=50000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

In [38]:
y_train_pred = regr.predict(X_train_n)
y_test_pred = regr.predict(X_test_n)

In [39]:
print("RMSE Train:",RMSE(y_train_pred,y_train_log))
print("RMSE Test:",RMSE(y_test_pred,y_test_log))

RMSE Train: 0.4312768246257255
RMSE Test: 0.43007780966372344


In [40]:
serie_pred = pd.Series(data=y_test_pred, index= y_test.index, name='target').reset_index()